In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.2'

os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-11 17:58:34--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.3’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.09s   

2022-11-11 17:58:34 (10.1 MB/s) - ‘postgresql-42.2.9.jar.3’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("mod22").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [ ]:
df.count()

9002021

In [5]:
review_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date", "product_title", "star_rating", "helpful_votes", "total_votes", "vine"])
review_df.show()

+--------------+-----------+----------+--------------+-----------+--------------------+-----------+-------------+-----------+----+
|     review_id|customer_id|product_id|product_parent|review_date|       product_title|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+----------+--------------+-----------+--------------------+-----------+-------------+-----------+----+
|R3W4P9UBGNGH1U|   16414143|B00YL0EKWE|     852431543| 2015-08-31|LG G4 Case Hard T...|          2|            1|          3|   N|
|R15V54KBMTQWAY|   50800750|B00XK95RPQ|     516894650| 2015-08-31|Selfie Stick Fibl...|          4|            0|          0|   N|
| RY8I449HNXSVF|   15184378|B00SXRXUKO|     984297154| 2015-08-31|Tribe AB40 Water ...|          5|            0|          0|   N|
|R18TLJYCKJFLSR|   10203548|B009V5X1CE|     279912704| 2015-08-31|RAVPower® Element...|          5|            0|          0|   N|
|R1NK26SWS53B8Q|     488280|B00D93OVF0|     662791300| 2015-08-31|Fosmon Micro USB 

In [ ]:
review_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date", "product_title", "star_rating", "helpful_votes", "total_votes", "vine"])
review_df.show()

Row(count(DISTINCT customer_id)=5197905)

In [6]:
custDF = review_df.groupBy("customer_id").count()


In [ ]:
custDF = custDF.na.drop(subset=["customer_id"])
custDF.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   46909180|    6|
|   42560427|    7|
|   43789873|    3|
|   22037526|    2|
|   34220092|    2|
|   42801586|    1|
|    9565734|    2|
|   15829398|    1|
|   38247118|    1|
|   32478248|    2|
|   48114630|    1|
|   23085063|    1|
|   32787070|    3|
|   43515569|    1|
|    4919528|    2|
|    5088547|    2|
|   41852407|    3|
|   49703087|    1|
|   12713799|    1|
|   36728141|    8|
+-----------+-----+
only showing top 20 rows



In [ ]:
custDF = custDF.withColumnRenamed("customer_id","PKcustomer_id")
custDF.show(truncate=False)

In [ ]:
custDF.count()

In [7]:
from pyspark.sql.functions import to_date
# Review DataFrame
review_id_df = review_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3W4P9UBGNGH1U|   16414143|B00YL0EKWE|     852431543| 2015-08-31|
|R15V54KBMTQWAY|   50800750|B00XK95RPQ|     516894650| 2015-08-31|
| RY8I449HNXSVF|   15184378|B00SXRXUKO|     984297154| 2015-08-31|
|R18TLJYCKJFLSR|   10203548|B009V5X1CE|     279912704| 2015-08-31|
|R1NK26SWS53B8Q|     488280|B00D93OVF0|     662791300| 2015-08-31|
|R11LOHEDYJALTN|   13334021|B00XVGJMDQ|     421688488| 2015-08-31|
|R3ALQVQB2P9LA7|   27520697|B00KQW1X1C|     554285554| 2015-08-31|
|R3MWLXLNO21PDQ|   48086021|B00IP1MQNK|     488006702| 2015-08-31|
|R2L15IS24CX0LI|   12738196|B00HVORET8|     389677711| 2015-08-31|
|R1DJ8976WPWVZU|   15867807|B00HX3G6J6|     299654876| 2015-08-31|
|R3MRWNNR8CBTB7|    1972249|B00U4NATNQ|     577878727| 2015-08-31|
|R1DS6DKTUXAQK3|   10956619|B00SZEFDH8|     654620704| 2015-08

In [ ]:
review_id_df = review_id_df.na.drop(subset=["review_id"])
review_id_df.show()

In [ ]:
review_id_df.count()

In [ ]:
products_df = review_df.select(["product_id", "product_title"])
products_df.show()

In [ ]:
products_df = products_df.groupBy("product_id","product_title").count()
products_df.show()

In [ ]:
products_df = products_df.na.drop(subset=["product_id"])
products_df.show()

In [ ]:
products_df = products_df.withColumnRenamed("product_id","PKproduct_id")
products_df.show()

In [ ]:
products_df.count()

In [ ]:
vine_df=review_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

In [ ]:
vine_df = vine_df.withColumnRenamed("review_id","PKreview_id")
vine_df.show()

In [ ]:
vine_df.count()

In [10]:
# Configuration for RDS instance

mode = "append"
jdbc_url="jdbc:postgresql://<>:5432/<>"
config = {"user":"<>", 
          "password": "<>", 
          "driver":"org.postgresql.Driver"}

In [ ]:
review_id_df.write.jdbc(url=jdbc_url, table='customer_id', mode=mode, properties=config)

In [ ]:
review_id_df.write.jdbc(url=jdbc_url, table='product_id', mode=mode, properties=config)

In [ ]:
review_id_df.write.jdbc(url=jdbc_url, table='product_parent', mode=mode, properties=config)

In [ ]:
review_id_df.write.jdbc(url=jdbc_url, table='review_date', mode=mode, properties=config)

In [ ]:
products_df.write.jdbc(url=jdbc_url, table='PKproduct_id', mode=mode, properties=config)

In [ ]:
products_df.write.jdbc(url=jdbc_url, table='product_title', mode=mode, properties=config)

In [ ]:
custDF.write.jdbc(url=jdbc_url, table='PKcustomer_id', mode=mode, properties=config)

In [ ]:
custDF.write.jdbc(url=jdbc_url, table='count', mode=mode, properties=config)

In [ ]:
vine_df.write.jdbc(url=jdbc_url, table='PKreview_id', mode=mode, properties=config)

In [ ]:
vine_df.write.jdbc(url=jdbc_url, table='star_rating', mode=mode, properties=config)

In [ ]:
vine_df.write.jdbc(url=jdbc_url, table='helpful_votes', mode=mode, properties=config)

In [ ]:
vine_df.write.jdbc(url=jdbc_url, table='total_votes', mode=mode, properties=config)

In [ ]:
vine_df.write.jdbc(url=jdbc_url, table='vine', mode=mode, properties=config)